In [1]:
from __future__ import print_function

Load in the Python modules necessary to load the data (gzip and yaml), process the data (numpy and pandas), and plot the data (toyplot).

In [2]:
import gzip
import yaml
import numpy
import pandas
import toyplot.pdf

print("yaml version:    ", yaml.__version__)
print("numpy version:   ", numpy.__version__)
print("pandas version:  ", pandas.__version__)
print("toyplot version: ", toyplot.__version__)

yaml version:     3.12
numpy version:    1.14.0
pandas version:   0.22.0
toyplot version:  0.16.0


## Ingest Data

Load the data, which is output into a YAML file. (Actually, there are several runs that have been concatinated into a YAML file that has a sequence in its top level.)

In [3]:
filename = 'miniGraphics-skybridge-vn-scaling.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data = pandas.DataFrame(yaml_data)

The YAML data is hierarchical. The basic yaml reader to DataFrame just embeds dictionaries and lists in DataFrame columns. Fix that by expanding the data of these columns into new columns.

In [4]:
def expand_single_column(original_data, column_to_expand):

    expanded_data = pandas.DataFrame()
    for index in original_data.index:
        sub_table = pandas.DataFrame(original_data[column_to_expand][index])
        for column in original_data.columns:
            if column != column_to_expand:
                sub_table[column] = numpy.full(sub_table.index.shape,
                                               original_data[column][index],
                                               dtype=original_data[column].dtype)
        expanded_data = expanded_data.append(sub_table, ignore_index=True)
    return expanded_data

def flatten_table(original_data):
    flat_data = original_data
    for column_name in original_data.columns:
        if isinstance(flat_data[column_name][0], list):
            flat_data = expand_single_column(flat_data, column_name)
    return flat_data

In [5]:
data = flatten_table(data)

Add a column that gives a human-readable name to each image resolution.

In [6]:
image_height_names = {
    500: 'Desktop Window',
    1080: 'HDTV',
    4320: '8K UHD',
}

data['image-size'] = data['image-height'].map(image_height_names)

Rename the algorithms from the identifiers the program wrote out to the strings used in the paper. Note that there are some extras in the data that we are ignoring.

In [7]:
algorithm_names = {
    '2-3-SwapBase': '2-3 Swap',
    'BinarySwapFold': 'Naive',
    'BinarySwapTelescoping': 'Telescoping',
    'BinarySwapRemainder': 'Remainder',
    'IceTBase': 'IceT'
}

data['composite-algorithm'] = data['composite-algorithm'].map(algorithm_names)

Print a summary of the table data. There are multiple ways that Jupyter and pandas will report a summary of a table, but I find this method the most effective. It prints out every column. Then for all columns with a "small" number of unique values, it gives those values. This latter information really helps identify the proper way to group values.

In [8]:
import IPython.display

data_description = ''

for column_name in data.columns:
    data_description = data_description + '**' + column_name + '**: '
    unique_values = data[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    else:
        data_description = (
            data_description +
            str(numpy.min(unique_values)) + ' &ndash; ' +
            str(numpy.max(unique_values)) + ' '
        )
    data_description = data_description + ' \n'
    
IPython.display.display(IPython.display.Markdown(data_description))

C:\Users\kmorel\AppData\Local\Continuum\Anaconda2\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
C:\Users\kmorel\AppData\Local\Continuum\Anaconda2\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


**color-buffer-format**: byte  
**composite-algorithm**: Naive Remainder Telescoping nan 2-3 Swap IceT  
**composite-seconds**: 0.00227687 &ndash; 28.7058  
**compress-seconds**: 2.315e-06 &ndash; 8e-06  
**construct-tree-seconds**: nan &ndash; nan  
**depth-buffer-format**: float  
**gather-seconds**: 0.00095674 &ndash; 4.52609  
**geometry**: box  
**geometry-distribution**: duplicate  
**geometry-overlap**: -0.05  
**icet-copy-result-seconds**: 1.304e-06 &ndash; 5e-05  
**image-compression**: True False  
**image-height**: 500 1080 4320  
**image-width**: 500 1920 7680  
**k**: nan &ndash; 8,8,8,8  
**max-image-split**: nan 1000000.0  
**num-processes**: 64 &ndash; 8192  
**num-triangles**: 768 &ndash; 98304  
**paint-seconds**: 0.00020071 &ndash; 0.170633  
**painter**: simple  
**partial-composite-seconds**: 0.00037141 &ndash; 27.0723  
**phi-rotation**: -178.406 &ndash; 136.756  
**random-seed**: 17627  
**rendering-order-dependent**: False  
**start-time**: 2018-04-03T12:58:47.000000000 &ndash; 2018-04-18T10:04:49.000000000  
**theta-rotation**: -177.945 &ndash; 179.418  
**total-seconds**: 0.00280725 &ndash; 28.7062  
**trial-num**: 0 &ndash; 19  
**uncompress-seconds**: 1.337e-06 &ndash; 8e-06  
**zoom**: 1.5  
**image-size**: Desktop Window HDTV 8K UHD  


## Plot Data

We are plotting the time it takes to do a "partial composite" (that is the time to blend all the pixels, but the pixels are left distributed across all the processes).

The first thing we want to do is to average the time it took over all trials. This is easily done with a pivot table.

In [9]:
average_partial_composite = data.pivot_table(
    values='partial-composite-seconds',
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

print(average_partial_composite.index)
print(average_partial_composite.columns)

Int64Index([  64,   96,  128,  144,  192,  256,  288,  384,  432,  512,  528,
             576, 1024, 1056, 1152, 1296, 1536, 1584, 1728, 2048, 2112, 2304,
            2592, 3072, 3168, 3456, 3888, 4096, 4608, 5184, 6144, 6912, 7776,
            8192],
           dtype='int64', name=u'num-processes')
MultiIndex(levels=[[u'8K UHD', u'Desktop Window', u'HDTV'], [u'2-3 Swap', u'IceT', u'Naive', u'Remainder', u'Telescoping']],
           labels=[[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2], [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4]],
           names=[u'image-size', u'composite-algorithm'])


Make a grouping structure of the data so we can pull out the actual data values for each trial.

In [10]:
grouped_data = data.groupby(['image-size', 'composite-algorithm'])

Plot data.

In [11]:
image_size = 'HDTV'
algorithms = ['2-3 Swap', 'Naive', 'Remainder', 'Telescoping', 'IceT']

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[64, 128, 256, 512, 1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

for algorithm in algorithms:
    # Plot line following average
    data_series = average_partial_composite[image_size, algorithm]
    x = data_series.index
    y = numpy.array(data_series)
    axes.plot(x, y)
    
    # Some adjustments for correct label placement
    if algorithm == 'Naive':
        vert_align = 'top'
    else:
        vert_align = 'middle'
    
    # Put label at end of series
    axes.text(
        x[-1],
        y[-1],
        algorithm,
        style={
            'text-anchor':'start',
            '-toyplot-anchor-shift':'2px',
            '-toyplot-vertical-align': vert_align,
        },
    )
    
    # Put a dot at every recorded sample.
    scatter_data = data.loc[grouped_data.groups[(image_size, algorithm)]]
    x = scatter_data['num-processes']
    y = scatter_data['partial-composite-seconds']
    axes.scatterplot(x, y, opacity=1)

<path d="M 45.0 206.02095260000002 L 71.36065221802102 208.02527659999998 L 90.06383261409567 207.20894793333332 L 97.72130443604203 207.04794193333333 L 116.42448483211666 207.95653313333332 L 135.12766522819135 206.97350453333334 L 142.7851370501377 207.2003996 L 161.48831744621233 207.98877893333332 L 169.14578926815864 206.40194066666666 L 180.191497842287 205.91987413333334 L 182.1920670061065 206.06101826666668 L 187.8489696642333 206.390419 L 225.25533045638267 204.22590666666667 L 227.25589962020217 204.17192580000003 L 232.91280227832897 203.90923675555555 L 240.57027410027533 202.92980836666666 L 251.61598267440365 201.9838844888889 L 253.6165518382232 200.78067560000002 L 259.27345449635 199.27562877777777 L 270.31916307047834 197.3299636888889 L 272.3197322342979 197.4886604 L 277.9766348924246 196.25622116666665 L 285.63410671437094 194.24386916666666 L 296.6798152884993 190.36867180000002 L 298.6803844523189 188.2956192 L 304.33728711044563 185.77114770000003 L 311.994758932392 181.79640313333334 L 315.3829956845739 177.4230807555556 L 323.04046750652026 172.87687143333335 L 330.6979393284667 165.71504803333335 L 341.743647902595 152.43923363333337 L 349.40111972454133 138.39451523333338 L 357.0585915464876 122.85172000000001 L 360.44682829866963 103.3488759" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 2-3 Swap <path d="M 45.0 207.22806646666666 L 71.36065221802102 181.0470934 L 90.06383261409567 208.53993086666668 L 97.72130443604203 181.8655628666667 L 116.42448483211666 181.92886513333335 L 135.12766522819135 208.68442 L 142.7851370501377 182.44858893333333 L 161.48831744621233 182.5478388 L 169.14578926815864 182.94434766666666 L 180.191497842287 208.71842006666668 L 182.1920670061065 183.00550913333333 L 187.8489696642333 182.49122653333336 L 225.25533045638267 208.02092066666668 L 227.25589962020217 182.36945366666666 L 232.91280227832897 181.7259648666667 L 240.57027410027533 181.46630366666668 L 251.61598267440365 181.40698566666669 L 253.6165518382232 181.82845566666668 L 259.27345449635 181.14435113333334 L 270.31916307047834 207.43918304444443 L 272.3197322342979 182.70532139999997 L 277.9766348924246 181.39705473333333 L 285.63410671437094 180.7401116 L 296.6798152884993 180.46708031111112 L 298.6803844523189 180.41720759999998 L 304.33728711044563 179.8762542 L 311.994758932392 179.22173433333336 L 315.3829956845739 205.9378974888889 L 323.04046750652026 179.76867900000002 L 330.6979393284667 178.8442696666667 L 341.743647902595 176.38778343333337 L 349.40111972454133 174.13657613333336 L 357.0585915464876 171.02824603333335 L 360.44682829866963 201.3186854" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Naive <g class="toyplot-Datum" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:1.0;opacity:1.0;stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0" tra

In [12]:
toyplot.pdf.render(canvas, 'scaling-hdtv.pdf')

In [13]:
image_size = '8K UHD'
algorithms = ['2-3 Swap', 'Naive', 'Remainder', 'Telescoping', 'IceT']

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[64, 128, 256, 512, 1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

for algorithm in algorithms:
    # Plot line following average
    data_series = average_partial_composite[image_size, algorithm]
    x = data_series.index
    y = numpy.array(data_series)
    axes.plot(x, y)
    
    if algorithm == 'Naive':
        vert_shift = '70%'
    elif algorithm == 'Remainder':
        vert_shift = '-80%'
    else:
        vert_shift = '0'
    
    # Put label at end of series
    axes.text(
        x[-1],
        y[-1],
        algorithm,
        style={
            'text-anchor':'start',
            '-toyplot-anchor-shift':'2px',
            'baseline-shift': vert_shift,
        },
    )
    
    # Put a dot at every recorded sample.
    scatter_data = data.loc[grouped_data.groups[(image_size, algorithm)]]
    x = scatter_data['num-processes']
    y = scatter_data['partial-composite-seconds']
    axes.scatterplot(x, y, opacity=1)

<path d="M 45.0 131.98692624486134 L 71.36065221802102 131.86554659542375 L 90.06383261409567 134.59665740804638 L 97.72130443604203 132.10849825507455 L 116.42448483211666 134.31080929542472 L 135.12766522819135 135.21452021944654 L 142.7851370501377 133.17912337214233 L 161.48831744621233 135.77044286108622 L 169.14578926815864 133.67093491526762 L 180.191497842287 135.1795655184992 L 182.1920670061065 134.9184769857153 L 187.8489696642333 122.18680424795687 L 225.25533045638267 135.04832050928164 L 227.25589962020217 135.5023194199355 L 232.91280227832897 134.92346920584532 L 240.57027410027533 133.8423833265968 L 251.61598267440365 133.1507180976041 L 253.6165518382232 132.9561772331595 L 259.27345449635 133.59436616652044 L 270.31916307047834 134.82304772091706 L 272.3197322342979 134.5933597947495 L 277.9766348924246 123.20469503238463 L 285.63410671437094 133.4498436837643 L 296.6798152884993 132.98898848546386 L 298.6803844523189 129.8326229580922 L 304.33728711044563 129.18308931898616 L 311.994758932392 131.67773377810857 L 315.3829956845739 131.2272752217304 L 323.04046750652026 130.66804580675694 L 330.6979393284667 124.5589835932648 L 341.743647902595 126.73014592801444 L 349.40111972454133 121.58677602847087 L 357.0585915464876 120.6764011775782 L 360.44682829866963 111.46876410974298" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 2-3 Swap <path d="M 45.0 133.91512317990475 L 71.36065221802102 48.58561434683597 L 90.06383261409567 137.18696013289036 L 97.72130443604203 52.32367885978493 L 116.42448483211666 53.33851935191299 L 135.12766522819135 137.81771111625935 L 142.7851370501377 52.42516290899769 L 161.48831744621233 54.59012847876022 L 169.14578926815864 53.98575840176661 L 180.191497842287 138.25439755710443 L 182.1920670061065 52.40018348827349 L 187.8489696642333 53.197436463026314 L 225.25533045638267 135.78662864635191 L 227.25589962020217 54.01101262359886 L 232.91280227832897 53.74719439980815 L 240.57027410027533 53.67833840216099 L 251.61598267440365 53.32908451386902 L 253.6165518382232 53.6814436546822 L 259.27345449635 53.718202882905985 L 270.31916307047834 137.39372048660746 L 272.3197322342979 55.65531253388804 L 277.9766348924246 53.83636277936061 L 285.63410671437094 53.635002267417235 L 296.6798152884993 52.50348122479967 L 298.6803844523189 53.77792632372805 L 304.33728711044563 53.826854661021116 L 311.994758932392 45.00770391967427 L 315.3829956845739 136.83449107163398 L 323.04046750652026 54.90083235160739 L 330.6979393284667 53.87553567731699 L 341.743647902595 52.946103369579134 L 349.40111972454133 52.85577624535429 L 357.0585915464876 52.145387900864556 L 360.44682829866963 132.20122988899513" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Naive <g class="toyplot-Datum" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacit

In [14]:
toyplot.pdf.render(canvas, 'scaling-8k.pdf')